In [106]:
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import cv2 
import matplotlib.pyplot as plt

In [107]:
def parse_cfg(cfgfile):
    
    file=open(cfgfile, 'r')
    lines=file.read().split('\n')
    lines=[x for x in lines if len(x)>0] #remove empty lines
    lines=[x for x in lines if x[0] !='#'] #remove commented lines
    lines = [x.rstrip().lstrip() for x in lines]
    
    block={}
    blocks=[]
    
    for line in lines:
        if line[0]=='[':
            if len(block) != 0:
                blocks.append(block)
                block = {}
            block["type"] = line[1:-1].rstrip()
            #print(block)
            
        else:
            key, value=line.split('=')
            block[key.rstrip()] = value.lstrip()
    blocks.append(block)
        
    return(blocks)

In [108]:
parse_cfg('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3_face.cfg')

[{'angle': '0',
  'batch': '64',
  'burn_in': '1000',
  'channels': '3',
  'decay': '0.0005',
  'exposure': '1.5',
  'height': '416',
  'hue': '.1',
  'learning_rate': '0.001',
  'max_batches': '500200',
  'momentum': '0.9',
  'policy': 'steps',
  'saturation': '1.5',
  'scales': '.1,.1',
  'steps': '400000,450000',
  'subdivisions': '16',
  'type': 'net',
  'width': '416'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '32',
  'pad': '1',
  'size': '3',
  'stride': '1',
  'type': 'convolutional'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '64',
  'pad': '1',
  'size': '3',
  'stride': '2',
  'type': 'convolutional'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '32',
  'pad': '1',
  'size': '1',
  'stride': '1',
  'type': 'convolutional'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '64',
  'pad': '1',
  'size': '3',
  'stride': '1',
  'type': 'convolutional'},
 {'activation': 'linear', 'from': '-3', 'type'

In [109]:
def get_test_input():
    img = cv2.imread("C:/Users/tunes/Downloads/test1.jpg")
    img = cv2.resize(img, (416,416)) 
    img_ =  img[:,:,::-1].transpose((2,0,1))
    img_ = img_[np.newaxis,:,:,:]/255.0
    img_ = torch.from_numpy(img_).float()
    img_ = Variable(img_)
    return img_

In [110]:
class Maxpoolstride1(nn.Module):
    
    def __inti__(self, kernel_size):
        super(Maxpoolstride1, self).__init__()
        self.kernel_size=kernel_size
        self.pad=kernel_size - 1
        
    def forward(self, x):
        padded_x=F.pad(x, (0,self.pad,0,self.pad), mode="replicate")
        pooled_x=nn.MaxPool2d(self.kernel_size, self.pad)(padded_x)
        
        return pooled_x

In [111]:
class Emptylayer(nn.Module):
    
    def __init__(self):
        super(Emptylayer, self).__init__()
       
    def forward(self, x):
        return x

In [112]:
class Detectionlayer(nn.Module):
    
    def __init__(self, anchors):
        super().__init__()
        self.anchors=anchors
        
    def forward(self, x, inp_dim, classes, confidence):
        x=x.data
        global CUDA
        prediction=x
        prediction=predict_transform(prediction, inp_dim, self.anchors, classes, confidence, CUDA)
        return prediction
        

In [113]:
class Upsample(nn.Module):
    
    def __init__(self, stride=2):
        super().__init__()
        self.stride=stride
        
    def forward(self, x):
        stride= self.stride
        assert(x.data.dim() == 4)
        B=x.data.size[0]
        C=x.data.size[1]
        H=x.data.size[2]
        W=x.data.size[3]
        ws=stride
        hs=stride
        x=x.view(B, C, H, 1, W, 1).expand(B, C, H, stride, W, stride).contiguous().view(B, C, H*stride, W*stride)
        
        return x

In [114]:
def Create_modules(blocks):
    
    net_info=blocks[0]
    
    module_list = nn.ModuleList()
    
    index = 0
    
    prev_filters = 3
    
    output_filters = []
    
    for x in blocks:
        module=nn.Sequential()
        
        if x['type'] == 'net':
            continue
            
        if x['type'] == 'convolutional':
            
            activation = x['activation']
            filters = int(x['filters'])
            padding = int(x['pad'])
            kernel_size = int(x['size'])
            stride = int(x['stride'])
            
            try:
                batch_normalize = x['batch_normalize']
                bias = False
                
            except:
                batch_normalize=0
                bias=True
            
            if padding:
                pad = (kernel_size - 1)//2
            else:
                pad = 0
                
            conv=nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias= bias)
            module.add_module("conv_{0}".format(index), conv)
            
            
            if batch_normalize:
                bn=nn.BatchNorm2d(filters)
                module.add_module("batch_norm_{0}".format(index), bn)
                
                
            if activation=='leaky':
                activ=nn.LeakyReLU(0.1, inplace=True)
                module.add_module("leaky_{0}".format(index), activ)
                
                
        elif x['type']=='Upsample':
            stride=int(x['stride'])
            upsample=Upsample(stride)
            upsample=nn.Upsample(scale_factor=2, mode='nearest')
            module.add_module("upsample_{0}".format(index), upsample)
            
            
        elif x['type']=='route':
            layers=x['layers'].split(',')
            
            #start of a route
            
            start=int(layers[0])
            
            #end, if exists
            try:
                end=int(layers[1])
                
            except:
                end=0
                
            if start > 0:
                start=start-index
                
            if end > 0:
                end=end-index
                
            route=Emptylayer()
            module.add_module("route_{0}".format(index), route)
            
            
            if end < 0:
                filters=output_filters[index + start] + output_filters[index+end]
                
            else:
                filters=output_filters[index + start]
                
            
        elif x['type']=='shortcut':
            from_=x['from']
            shortcut=Emptylayer()
            module.add_module("shortcut_{}".format(index), shortcut)
            
            
        elif x['type']=='maxpool':
            
            stride = int(x["stride"])
            size = int(x["size"])
            
            if stride != 1:
                
                maxpool = nn.MaxPool2d(size, stride)
                
            else:
                
                maxpool = Maxpoolstride1(size)
            
            module.add_module("maxpool_{}".format(index), maxpool)
                
                
            
            
        elif x['type']=='yolo':
            mask=x['mask'].split(',')
            mask=[int(x) for x in mask]
            
            anchors=x['anchors'].split(',')
            anchors=[int(a) for a in anchors]
            anchors=[(anchors[i], anchors[i+1]) for i in range(0, len(anchors), 2)]
            anchors=[anchors[i] for i in mask]
            
            
            detection=Detectionlayer(anchors)
            module.add_module('detection_{}'.format(index), detection)
            
            
            
            
        
            
            
        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
        index += 1
        
    
    return (net_info, module_list)
            
               

In [145]:
class Darknet(nn.Module):
    
    def __init__(self, cfgfile):
        super(Darknet, self).__init__()
        self.blocks = parse_cfg(cfgfile)
        self.net_info, self.module_list = Create_modules(self.blocks)
        self.header = torch.IntTensor([0,0,0,0])
        self.seen = 0

        
        
    def get_blocks(self):
        return self.blocks

    
    def get_module_list(self):
        return self.module_list
    
    
    
    def forward(self, x, CUDA = False):
        
        
        detections=[]
        
        modules = self.blocks[1:]
        outputs={}
        
        
        write=0
    
        
        for i in range(len(modules)):
            #print(len(modules))
            modules_type = (modules[i]['type'])
        
            if modules_type == 'convolution' or modules_type == 'upsample' or modules_type == 'maxpool' :
                x=self.modules_list[i](x)
                print(x)
                outputs[i]=x
                print(outputs[i])
            elif modules_type == 'route':
                layers = modules[i]['layers']
                layers = [int(a) for a in layers]
                
                if layers[0] > 0:
                    layers[0] = layers[0] - i
                    
                if len(layers) == 1:
                    x =  outputs[i + (layers[0])]
                    
                else:
                    if layers[1] > 0:
                        layers[1] = layers[1] - i

                    map1 = outputs [i + layers[0]]
                    map2 = outputs [i + layers[1]]
                    
                    x = torch.cat((map1, map2), 1)
                
                outputs[i] = x
                
            elif modules_type == 'shortcut':
                from_ = int(modules[i]['from'])
                print(from_)
                print(i)
                print(outputs[i-1])
                x = outputs[i-1] + outputs[i + from_]
                outputs[i] = x
                
            
            elif modules_type == 'yolo':
                
                anchors=self.module_list[i][0].anchors
                
                inp_dim=int(self.net_info['height'])
                
                num_classes=int(module[i]['classes'])
                
                x=x.data
                x=predict_transform(x, inp_dim, anchors, num_classes, CUDA)
                
                if type(x) == int:
                    continue
                    
                if not write:
                    detections = x
                    write = 1 
                    
                else:
                    detections = torch.cat((detection, x), 1)
                    
                outputs[i] = outputs[i-1]
                
                
                
        try:
            return detections
        
        except:
            return 0
        
                 
    def load_weights(self, weightfile):
        
        fp = open(weightfile, 'rb')
        
        
        header = np.fromfile(fp, dtype = np.int32, count = 5)
        self.header = torch.from_numpy(header)
        self.seen = self.header[3]
            
        
        weights = np.fromfile(fp, dtype = np.float32)
        
        ptr = 0
        
        
        
        for i in range(len(self.module_list)):
            
            module_type = self.blocks[i+1]['type']
            
            if module_type == 'convolutional':
                model = self.module_list[i]
                
                try:
                    batch_normalize = int(self.blocks[i+1]['batch_normalize'])
                    
                except:
                    batch_normalize = 0
                    
                
                conv = model[0]
                
                if batch_normalize:
                    
                    bn = model[1]
                    
                    num_bn_biases = bn.bias.numel()
                    
                    bn_biases = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_weights = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_running_mean = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_running_var = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_biases = bn_biases.view_as(bn.bias.data)
                    bn_weights = bn_weights.view_as(bn.weight.data)
                    bn_running_mean = bn_running_mean.view_as(bn.running_mean)
                    bn_running_var = bn_running_var.view_as(bn.running_var)
                    
                    
                    bn.bias.data.copy_(bn_biases)
                    bn.weight.data.copy_(bn_weights)
                    bn.running_mean.copy_(bn_running_mean)
                    bn.running_var.copy_(bn_running_var)
                    
                    
                else:
                    num_biases = conv.bias.numel()
                    
                    conv_biases = torch.from_numpy(weights[ptr:ptr  + num_biases])
                    ptr += num_biases
                    
                    conv_biases = conv_biases.view_as(conv.bias.data)
                    
                    conv.bias.data.copy_(conv_biases)
                    
                
                num_weights = conv.weight.numel()
                
                conv_weights = torch.from_numpy(weights[ptr:ptr + num_weights])
                ptr += num_weights
                #print(conv_weights.shape)
                conv_weights = conv_weights.view_as(conv.weight.data)
                
                conv.weight.data.copy_(conv_weights)
                
                
                    
    def save_weights(self, savedfile, cutoff = 0):
        
        if cutoff <= 0:
            cutoff = len(self.blocks) - 1
            
        fp = open(savedfile, 'wb')
        
        self.header[3] = self.seen
        header = self.header
        
        header = header.numpy()
        header.tofile(fp)
        
        
        for i in range(len(self.module_list)):
            module_type = self.block[i+1]['type']
            
            
            if module_type == 'convolutional':
                model = self.module_list[i]
                
                try:
                    batch_normalize = int(self.block[i+1]['batch_normalize'])
                    
                except:
                    batch_normalize = 0
                
                conv = model[0]
                
                if batch_normalize:
                    bn = model[1]
                    
                    
                    cpu(bn.bias.data).numpy().tofile(fp)
                    cpu(bn.weights.data).numpy().tofile(fp)
                    cpu(bn.running_mean).numpy().tofile(fp)
                    cpu(bn.running_var).numpy().tofile(fp)
                    
                else:
                    cpu(conv.bias.data).numpy().tofile(fp)
                    
                
                cpu(conv.weights.data).numpy().tofile(fp)
                
                
            
                    

In [146]:
a = Darknet('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3_face.cfg')

In [147]:
a.load_weights('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3.weights')

In [148]:
inp = get_test_input()

In [149]:
s,d = a(inp)

-3
4


KeyError: 3

In [120]:
a.eval()

Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1, inplace)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1, inplace)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1, inplace)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_3): Batch

In [ ]:
outputs[3]

In [140]:
class Darknet(nn.Module):
    def __init__(self, cfgfile):
        super(Darknet, self).__init__()
        self.blocks = parse_cfg(cfgfile)
        self.net_info, self.module_list = Create_modules(self.blocks)
        self.header = torch.IntTensor([0,0,0,0])
        self.seen = 0

        
        
    def get_blocks(self):
        return self.blocks
    
    def get_module_list(self):
        return self.module_list

                
    def forward(self, x, CUDA=False):
        detections = []
        modules = self.blocks[1:]
        outputs = {}   #We cache the outputs for the route layer
        
        
        write = 0
        for i in range(len(modules)):        
            
            module_type = (modules[i]["type"])
            if module_type == "convolutional" or module_type == "upsample" or module_type == "maxpool":
                
                x = self.module_list[i](x)
                outputs[i] = x
                print(outputs[i])
                
            elif module_type == "route":
                layers = modules[i]["layers"]
                layers = [int(a) for a in layers]
                
                if (layers[0]) > 0:
                    layers[0] = layers[0] - i

                if len(layers) == 1:
                    x = outputs[i + (layers[0])]

                else:
                    if (layers[1]) > 0:
                        layers[1] = layers[1] - i
                        
                    map1 = outputs[i + layers[0]]
                    map2 = outputs[i + layers[1]]
                    
                    
                    x = torch.cat((map1, map2), 1)
                outputs[i] = x
            
            elif  module_type == "shortcut":
                from_ = int(modules[i]["from"])
                x = outputs[i-1] + outputs[i+from_]
                outputs[i] = x
                
            
            
            elif module_type == 'yolo':        
                
                anchors = self.module_list[i][0].anchors
                #Get the input dimensions
                inp_dim = int (self.net_info["height"])
                
                #Get the number of classes
                num_classes = int (modules[i]["classes"])
                
                #Output the result
                x = x.data
                x = predict_transform(x, inp_dim, anchors, num_classes, CUDA)
                
                if type(x) == int:
                    continue

                
                if not write:
                    detections = x
                    write = 1
                
                else:
                    detections = torch.cat((detections, x), 1)
                
                outputs[i] = outputs[i-1]
                
        
        
        try:
            return detections
        except:
            return 0

            
    def load_weights(self, weightfile):
        
        #Open the weights file
        fp = open(weightfile, "rb")

        #The first 4 values are header information 
        # 1. Major version number
        # 2. Minor Version Number
        # 3. Subversion number 
        # 4. IMages seen 
        header = np.fromfile(fp, dtype = np.int32, count = 5)
        self.header = torch.from_numpy(header)
        self.seen = self.header[3]
        
        #The rest of the values are the weights
        # Let's load them up
        weights = np.fromfile(fp, dtype = np.float32)
        
        ptr = 0
        for i in range(len(self.module_list)):
            module_type = self.blocks[i + 1]["type"]
            
            if module_type == "convolutional":
                model = self.module_list[i]
                try:
                    batch_normalize = int(self.blocks[i+1]["batch_normalize"])
                except:
                    batch_normalize = 0
                
                conv = model[0]
                
                if (batch_normalize):
                    bn = model[1]
                    
                    #Get the number of weights of Batch Norm Layer
                    num_bn_biases = bn.bias.numel()
                    
                    #Load the weights
                    bn_biases = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_weights = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
                    
                    bn_running_mean = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
                    
                    bn_running_var = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
                    
                    #Cast the loaded weights into dims of model weights. 
                    bn_biases = bn_biases.view_as(bn.bias.data)
                    bn_weights = bn_weights.view_as(bn.weight.data)
                    bn_running_mean = bn_running_mean.view_as(bn.running_mean)
                    bn_running_var = bn_running_var.view_as(bn.running_var)

                    #Copy the data to model
                    bn.bias.data.copy_(bn_biases)
                    bn.weight.data.copy_(bn_weights)
                    bn.running_mean.copy_(bn_running_mean)
                    bn.running_var.copy_(bn_running_var)
                
                else:
                    #Number of biases
                    num_biases = conv.bias.numel()
                
                    #Load the weights
                    conv_biases = torch.from_numpy(weights[ptr: ptr + num_biases])
                    ptr = ptr + num_biases
                    
                    #reshape the loaded weights according to the dims of the model weights
                    conv_biases = conv_biases.view_as(conv.bias.data)
                    
                    #Finally copy the data
                    conv.bias.data.copy_(conv_biases)
                    
                    
                #Let us load the weights for the Convolutional layers
                num_weights = conv.weight.numel()
                
                #Do the same as above for weights
                conv_weights = torch.from_numpy(weights[ptr:ptr+num_weights])
                ptr = ptr + num_weights

                conv_weights = conv_weights.view_as(conv.weight.data)
                conv.weight.data.copy_(conv_weights)
                
    def save_weights(self, savedfile, cutoff = 0):
            
        if cutoff <= 0:
            cutoff = len(self.blocks) - 1
        
        fp = open(savedfile, 'wb')
        
        # Attach the header at the top of the file
        self.header[3] = self.seen
        header = self.header

        header = header.numpy()
        header.tofile(fp)
        
        # Now, let us save the weights 
        for i in range(len(self.module_list)):
            module_type = self.blocks[i+1]["type"]
            
            if (module_type) == "convolutional":
                model = self.module_list[i]
                try:
                    batch_normalize = int(self.blocks[i+1]["batch_normalize"])
                except:
                    batch_normalize = 0
                    
                conv = model[0]

                if (batch_normalize):
                    bn = model[1]
                
                    #If the parameters are on GPU, convert them back to CPU
                    #We don't convert the parameter to GPU
                    #Instead. we copy the parameter and then convert it to CPU
                    #This is done as weight are need to be saved during training
                    cpu(bn.bias.data).numpy().tofile(fp)
                    cpu(bn.weight.data).numpy().tofile(fp)
                    cpu(bn.running_mean).numpy().tofile(fp)
                    cpu(bn.running_var).numpy().tofile(fp)
                
            
                else:
                    cpu(conv.bias.data).numpy().tofile(fp)
                
                
                #Let us save the weights for the Convolutional layers
                cpu(conv.weight.data).numpy().tofile(fp)
               





In [141]:
v = Darknet('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3_face.cfg')

In [142]:
v.load_weights('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3.weights')

In [143]:
inp = get_test_input()

In [144]:
s,d = v(inp)

tensor([[[[-5.8882e-01, -4.4997e-01, -4.4997e-01,  ..., -5.0809e-01,
           -5.0809e-01, -7.1135e-01],
          [-5.7352e-01, -4.7806e-01, -4.7806e-01,  ..., -5.4145e-01,
           -5.4145e-01, -6.5045e-01],
          [-5.7352e-01, -4.7806e-01, -4.7806e-01,  ..., -5.4145e-01,
           -5.4145e-01, -6.5045e-01],
          ...,
          [-6.3449e-01, -6.1667e-01, -7.2394e-01,  ..., -3.5784e-01,
           -4.5045e-01, -4.7052e-01],
          [-6.6337e-01, -6.4690e-01, -7.7161e-01,  ..., -4.7893e-01,
           -4.4996e-01, -5.1638e-01],
          [ 3.5957e+00,  4.5302e+00,  4.7132e+00,  ...,  3.9828e+00,
            4.1806e+00,  3.6565e+00]],

         [[ 1.7955e+00,  1.3114e+00,  1.3114e+00,  ...,  1.1926e+00,
            1.1926e+00,  7.5540e-01],
          [ 8.3093e-01, -1.0679e-01, -1.0679e-01,  ..., -1.3527e-01,
           -1.3527e-01, -2.0244e-01],
          [ 8.3093e-01, -1.0679e-01, -1.0679e-01,  ..., -1.3527e-01,
           -1.3527e-01, -2.0244e-01],
          ...,
     

tensor([[[[-5.7650e-01, -4.3203e-01, -2.5474e-01,  ..., -2.6528e-01,
           -4.1470e-01, -3.3949e-01],
          [-7.8383e-01, -1.0505e-01,  3.2922e-01,  ...,  2.2844e-01,
            8.5964e-01, -2.5114e-01],
          [-9.5648e-01, -1.1966e-01,  1.2516e-01,  ...,  1.8131e-01,
            1.1817e+00, -1.2121e-01],
          ...,
          [-9.9658e-01, -1.7283e-01,  3.9855e-01,  ...,  8.5586e-01,
           -2.0488e-02, -6.1092e-02],
          [-9.2253e-01, -7.0162e-02,  1.4136e+00,  ..., -2.6020e-01,
            1.7583e-01,  1.2422e-01],
          [-6.9387e-01,  8.2176e-01,  8.4928e-01,  ..., -3.1115e-01,
            2.7862e+00, -5.9538e-02]],

         [[-3.0202e-02, -4.1234e-02, -3.6963e-02,  ..., -3.6978e-02,
           -4.1526e-02, -4.0469e-02],
          [-4.3657e-02, -6.5657e-02, -5.9313e-02,  ..., -6.0614e-02,
           -6.6180e-02, -5.5159e-02],
          [-2.5219e-02, -3.6952e-02, -3.4383e-02,  ..., -3.4489e-02,
           -4.0206e-02, -3.4493e-02],
          ...,
     

tensor([[[[-4.2600e-02, -4.2597e-02, -4.2596e-02,  ..., -4.2596e-02,
           -4.2596e-02, -4.2598e-02],
          [-4.2600e-02, -4.2591e-02, -4.2590e-02,  ..., -4.2591e-02,
           -4.2590e-02, -4.2593e-02],
          [-4.2603e-02, -4.2596e-02, -4.2595e-02,  ..., -4.2597e-02,
           -4.2596e-02, -4.2596e-02],
          ...,
          [-4.2602e-02, -4.2597e-02, -4.2600e-02,  ..., -4.2598e-02,
           -4.2598e-02, -4.2598e-02],
          [-4.2603e-02, -4.2598e-02, -4.2600e-02,  ..., -4.2598e-02,
           -4.2596e-02, -4.2599e-02],
          [-4.2601e-02, -4.2598e-02, -4.2599e-02,  ..., -4.2599e-02,
           -4.2599e-02, -4.2600e-02]],

         [[ 1.1728e-01, -1.4781e-01, -2.1588e-01,  ..., -2.3388e-01,
           -1.9953e-01, -7.4049e-02],
          [-1.4385e-01, -3.2297e-01, -3.0499e-01,  ..., -3.3141e-01,
           -3.1916e-01, -1.7557e-01],
          [ 1.9708e+00, -1.5638e-02,  9.6895e-02,  ...,  9.6020e-02,
           -2.8066e-03,  6.0443e-01],
          ...,
     

tensor([[[[-1.2788e-01, -7.7600e-02, -1.0591e-01,  ..., -1.1265e-01,
           -1.1812e-01, -1.0993e-01],
          [-1.4276e-02,  1.0240e+00,  9.1179e-02,  ...,  5.4977e-01,
            3.7657e-01,  7.7973e-02],
          [-3.2397e-02,  1.1213e+00,  2.1177e-01,  ...,  5.4165e-01,
            5.0898e-01,  2.4606e-01],
          ...,
          [-5.3809e-03,  1.1311e+00,  5.1657e-01,  ...,  3.9832e-01,
           -6.8571e-02,  1.2562e+00],
          [ 5.3977e-01,  6.6747e-01,  1.6850e+00,  ..., -4.9445e-02,
            3.0982e-02,  1.7654e+00],
          [-1.2843e-01,  1.2968e+00,  1.1591e+00,  ..., -7.6823e-02,
            1.1455e+00,  8.4822e-01]],

         [[-7.6675e-02,  3.8869e-02,  1.9400e-01,  ...,  2.6419e-01,
            2.8338e-01,  1.6577e+00],
          [-3.2431e-01, -7.1823e-02, -5.7909e-02,  ..., -5.0659e-02,
           -6.8208e-02,  8.3847e-01],
          [-3.2398e-01, -6.0506e-02, -2.3101e-02,  ..., -3.8660e-02,
           -4.3598e-02,  9.2977e-01],
          ...,
     

tensor([[[[ 3.0289e+00,  5.2452e+00,  4.8460e+00,  ...,  4.9301e+00,
            4.9398e+00,  4.4597e+00],
          [ 1.0767e+00,  8.3544e-01,  1.0890e-01,  ...,  2.3427e-02,
            1.1826e-01, -5.4637e-02],
          [ 5.2007e-01, -1.5467e-02, -5.2631e-02,  ..., -4.5715e-02,
           -3.6194e-02, -7.2010e-02],
          ...,
          [ 2.5707e-01, -9.7561e-02, -1.9646e-01,  ..., -8.3262e-02,
           -9.4104e-02, -1.2768e-01],
          [-1.0815e-02, -1.4983e-01, -1.5056e-01,  ..., -1.3236e-01,
           -1.0941e-01, -1.1916e-01],
          [-1.7845e-01, -2.1171e-01, -8.2393e-02,  ..., -1.4046e-01,
           -1.0757e-01, -2.2161e-01]],

         [[ 2.6343e+00,  2.9772e+00,  2.6691e+00,  ...,  2.8322e+00,
            2.9738e+00,  4.1315e+00],
          [-1.6827e-01, -2.2406e-02,  4.0090e-02,  ..., -5.1588e-03,
           -1.7364e-02, -4.0074e-02],
          [-1.6613e-01,  7.0051e-02, -1.2181e-02,  ..., -2.5794e-02,
           -2.1393e-02, -3.7378e-02],
          ...,
     

...

tensor([[[[-3.0542e-02, -3.0588e-02, -3.0564e-02,  ..., -3.0565e-02,
           -3.0565e-02, -3.0560e-02],
          [-3.0549e-02, -3.0572e-02, -3.0565e-02,  ..., -3.0566e-02,
           -3.0564e-02, -3.0561e-02],
          [-3.0559e-02, -3.0570e-02, -3.0565e-02,  ..., -3.0565e-02,
           -3.0565e-02, -3.0563e-02],
          ...,
          [-3.0562e-02, -3.0568e-02, -3.0563e-02,  ..., -3.0565e-02,
           -3.0565e-02, -3.0563e-02],
          [-3.0560e-02, -3.0569e-02, -3.0564e-02,  ..., -3.0564e-02,
           -3.0565e-02, -3.0562e-02],
          [-3.0559e-02, -3.0567e-02, -3.0563e-02,  ..., -3.0563e-02,
           -3.0564e-02, -3.0561e-02]],

         [[-3.0080e-02, -2.7863e-02, -2.8065e-02,  ..., -2.8140e-02,
           -2.8271e-02, -2.8513e-02],
          [-2.9596e-02, -2.7578e-02, -2.7855e-02,  ..., -2.8010e-02,
           -2.7981e-02, -2.8349e-02],
          [-2.8583e-02, -2.7931e-02, -2.7878e-02,  ..., -2.8052e-02,
           -2.7976e-02, -2.8089e-02],
          ...,
     

tensor([[[[ 1.0489e+00, -5.6942e-02,  3.2751e-01,  ..., -1.4751e-02,
            1.1539e-02,  1.3852e-01],
          [-1.3953e-01,  2.8774e-01, -4.7084e-02,  ..., -8.4606e-02,
           -1.6947e-01, -1.4364e-01],
          [-2.7037e-02, -8.8080e-02, -4.7476e-02,  ..., -6.9891e-02,
           -2.7679e-02,  8.4810e-03],
          ...,
          [-5.4300e-02, -6.8117e-02, -5.5993e-02,  ...,  1.1405e+00,
            2.0720e-01, -6.8724e-03],
          [-7.5362e-02, -5.7161e-03,  2.7521e-02,  ..., -1.6829e-02,
           -5.5258e-02, -8.4722e-02],
          [-1.7233e-02,  3.8528e-01,  4.8964e-02,  ..., -9.5954e-03,
           -9.5290e-02, -7.1539e-02]],

         [[-4.6870e-02, -4.6870e-02, -4.6870e-02,  ..., -4.6870e-02,
           -4.6870e-02, -4.6870e-02],
          [-4.6870e-02, -4.6870e-02, -4.6870e-02,  ..., -4.6870e-02,
           -4.6870e-02, -4.6870e-02],
          [-4.6870e-02, -4.6870e-02, -4.6870e-02,  ..., -4.6870e-02,
           -4.6870e-02, -4.6870e-02],
          ...,
     

tensor([[[[-4.9782e-01, -3.9229e-01, -1.9630e-01,  ..., -1.8918e-01,
            1.3303e+00,  4.1950e-01],
          [-1.4710e-01, -1.6978e-01, -1.4819e-01,  ..., -1.1526e-01,
            1.3559e+00, -1.6967e-01],
          [-1.0361e-01, -6.5466e-02,  1.8294e-01,  ..., -5.1074e-02,
            9.3520e-01, -8.8662e-02],
          ...,
          [ 1.0679e+00,  1.4663e+00,  5.7086e-01,  ..., -1.4064e-01,
           -1.2327e-01, -5.6309e-02],
          [ 6.5262e-01,  4.6694e-01, -2.5927e-02,  ..., -6.6461e-02,
           -7.3145e-02, -4.2268e-02],
          [ 6.0990e-01,  6.4811e-01,  4.2895e-02,  ...,  5.3250e-01,
            3.6178e-01,  1.8646e-01]],

         [[ 2.7876e+00, -1.7432e-01,  2.4163e+00,  ...,  1.6301e+00,
            4.5653e+00,  1.8402e+00],
          [ 7.8313e-02, -2.1206e-01,  2.1819e+00,  ..., -4.4557e-02,
            3.7131e+00,  1.1623e+00],
          [ 1.1264e+00, -1.7622e-02,  1.2567e+00,  ...,  1.0709e-02,
            1.6713e+00,  4.4696e-01],
          ...,
     

tensor([[[[-2.2105e-01, -1.6179e-01,  2.2444e+00,  ..., -4.1723e-02,
           -7.5843e-02,  8.7937e-01],
          [-2.7201e-01, -1.5782e-01,  3.1998e+00,  ..., -2.1926e-01,
           -2.7360e-01,  2.5104e+00],
          [ 6.6549e-01, -1.1780e-02,  2.4897e+00,  ..., -1.2720e-01,
            2.5346e-01,  2.3744e+00],
          ...,
          [-5.2743e-02,  1.0381e-01, -1.9756e-02,  ...,  3.7924e+00,
            2.3115e+00,  1.2725e-01],
          [-7.8023e-02, -6.1006e-02, -7.3012e-02,  ...,  3.6563e+00,
            1.6275e+00,  1.4185e-01],
          [-8.7246e-02, -6.4012e-02, -4.4447e-02,  ...,  1.9633e+00,
            8.8069e-01, -7.7484e-03]],

         [[ 1.6140e+00, -1.7991e-01, -3.6764e-01,  ...,  2.3774e+00,
            8.9340e-01,  4.6449e-01],
          [ 4.4090e+00, -1.6601e-01, -5.2119e-01,  ...,  9.4817e-01,
            1.7185e+00,  9.4948e-01],
          [ 2.9686e+00, -1.9450e-01, -3.3475e-01,  ...,  3.1584e-01,
           -1.8358e-02, -5.8823e-02],
          ...,
     

NameError: name 'predict_transform' is not defined